# Euler-Lagrangian of a Cartpole on a Wheel

In [1]:
from sympy import symbols, diff, simplify, Function
from sympy import sin, cos
import sympy

In [2]:
# Define symbols of variables that we need to model our simulated physics model.
t = symbols('t')
gravity = symbols('g')
l1 = symbols('l1') # length of pole
x1 = symbols('x1') # Distance of center of mass of the pole from the center of the wheel
m1 = symbols('m1') # mass of pole
I1 = symbols('I1') # Inertia
wheel_mass = symbols("m")
wheel_inertia = symbols("I_w")
r = symbols('r') # radius of wheel

wheel_pos = Function('w')(t)
wheel_vel = diff(wheel_pos, t)
wheel_acc = diff(wheel_vel, t)
theta1 = Function('theta_1')(t)
theta1_vel = diff(theta1, t)
theta1_acc = diff(theta1_vel, t)


In [6]:
# Define duplicate of symbols for better rendering where derivatives of time are displayed with a dot above the symbol
# Kind of a hack, can probably be simplified
wheel_pos_ = symbols('q')
wheel_vel_ = symbols("\\dot{q}")
wheel_acc_ = symbols("\\ddot{q}")
theta1_ = symbols("d_1")
theta1_vel_ = symbols("\\dot{d_1}")
theta1_acc_ = symbols("\\ddot{d_1}")
def remove_time(v):
    return v.subs([
        (wheel_acc, wheel_acc_),
        (wheel_vel, wheel_vel_),
        (wheel_pos, wheel_pos_),
        (theta1_acc, theta1_acc_),
        (theta1_vel, theta1_vel_),
        (theta1, theta1_),
    ])

In [7]:
# Test display:
x = theta1 + theta1_vel + theta1_acc + wheel_pos + wheel_vel + wheel_acc
x

theta_1(t) + w(t) + Derivative(theta_1(t), t) + Derivative(theta_1(t), (t, 2)) + Derivative(w(t), t) + Derivative(w(t), (t, 2))

In [8]:
remove_time(x)

\ddot{d_1} + \ddot{q} + \dot{d_1} + \dot{q} + d_1 + q

## Define Lagrangian

In [9]:
# This is the only thing we need to define to calculate the update matrix.
# It's Kinetic Energy minus Potential Energy
def sq(v):
    return v**2
def lagrange():
    kinE = 0
    kinE += wheel_mass * sq(wheel_vel*r)
    kinE += m1 * (sq(wheel_vel*r + x1*theta1_vel*cos(theta1)) + sq(x1*theta1_vel*sin(theta1)))
    kinE += wheel_inertia * sq(wheel_vel)
    kinE += I1 * sq(theta1_vel)
    kinE /= 2
    potE = gravity * (m1*x1*cos(theta1))

    return kinE-potE

In [10]:
remove_time(lagrange())

I1*\dot{d_1}**2/2 + I_w*\dot{q}**2/2 + \dot{q}**2*m*r**2/2 - g*m1*x1*cos(d_1) + m1*(\dot{d_1}**2*x1**2*sin(d_1)**2 + (\dot{d_1}*x1*cos(d_1) + \dot{q}*r)**2)/2

Now we use the automatic differentiation of sympy to do all the dirty work:

In [13]:
e1 = remove_time(simplify(diff(diff(lagrange(), wheel_vel), t) - diff(lagrange(), wheel_pos)))
e1

I_w*\ddot{q} + \ddot{q}*m*r**2 + m1*r*(\ddot{d_1}*x1*cos(d_1) + \ddot{q}*r - \dot{d_1}**2*x1*sin(d_1))

In [14]:
e2 = remove_time(simplify(diff(diff(lagrange(), theta1_vel), t) - diff(lagrange(), theta1)))
e2

I1*\ddot{d_1} + \ddot{d_1}*m1*x1**2 + \ddot{q}*m1*r*x1*cos(d_1) - g*m1*x1*sin(d_1)

In [15]:
a, b = sympy.linear_eq_to_matrix([e1, e2], [remove_time(x) for x in [wheel_acc, theta1_acc]])

And just like that we get a and b.

a and b is the system of linear equations that is solved at runtime to calculate the acceleration of the relevant variables (wheel position and pole angle in this case):

They are directly copied into code in `pendulum_simulation.cpp`

In [16]:
simplify(a)

Matrix([
[I_w + m*r**2 + m1*r**2, m1*r*x1*cos(d_1)],
[      m1*r*x1*cos(d_1),    I1 + m1*x1**2]])

In [17]:
simplify(b)

Matrix([
[\dot{d_1}**2*m1*r*x1*sin(d_1)],
[             g*m1*x1*sin(d_1)]])

In [21]:
# We can also see here that the determinate of Matrix a is never 0.
# That means it can always be solved.
simplify(a.det())

I1*I_w + I1*m*r**2 + I1*m1*r**2 + I_w*m1*x1**2 + m*m1*r**2*x1**2 + m1**2*r**2*x1**2*sin(d_1)**2